In [64]:
import pandas as ps
import numpy as np
import random

In [65]:
class Config():
    T = 1000  # time
    N = 100   # number of banks
    S = 10
    α = 0.3   # alpha, liquidation cost of collaterals
    φ = 0.1   # phi    attractiveness
    δ = 0.015 # delta  screening cost
    σ = 1.5   # sigma 
    depositRate = 0.03
    riskfree = 0.01
    RANDOM_CONNECTIVITY = 0.75
    TOT_SIMULATIONS = 4
    τ = 1     # tau
    η = 0.09  #  eta 
    prud = 0.045
    u = 1.0   #  ypsilon 

    # banks parameters
    initial_loan = 120.0
    initial_liquidity = 30.0
    initial_deposit = 130.0
    initial_equity = initial_liquidity / 2
    initial_probfail = 0.2
    initial_price = 0.3
    initial_haircut = 0.3
    minInterestInterbank = 0.02
    
class Status():
    maxEquity = 0.0
    banks = []
    matched = []
    credit = []
    interestInterbank = []
    marketBank = []
    
    mark = 0
    
    def init(self):
        self.maxEquity = 0.0
        self.banks = []
        self.matched = np.zeros( (Config.N,Config.N) )
        self.interestInterbank = np.zeros( (Config.N,Config.N) )  
        self.credit = np.zeros( (Config.N,Config.N) )  
        self.marketBank = np.zeros( (Config.N,Config.N) )  
        for i in range(Config.N):
            self.banks.append( Bank() )
        

In [53]:
class Bank():
    def set_capacity():
        self.capacity = 1.0 - self.haircut*self.asset
        if self.capacity<1.0:
            self.capacity = 4.0;
        # fprintf( out49 ) **
        
    def determineProbFail():
        self.probFail = 1.0 - self.equity / Status.maxEquity
            
    def __init__(self):
        self.loan = Config.initial_loan
        self.liquidity = Config.initial_liquidity
        self.deposit = Config.initial_deposit
        self.equity = Config.initial_equity
        self.probfail = Config.initial_probfail
        self.haircut = Config.initial_haircut
        self.new_deposit = self.Dloan  = 0.0
        self.interbankLoan = self.interbankDebt = self.failB = self.asset = self.leverage = 0.0
        self.deltaD = self.firesale = self.crunch =  0.0
        
        self.asset = self.loan+ self.liquidity
        self.leverage = self.loan / self.equity
        
        self.rate = 0.0       # tasso
        self.totalRate = 0.0  # sommatasso
        self.connected = 0.0  # collegati
        
        self.marked = 0
        self.neightbour = 0   
        
        self.a = 0.0
        self.b = 0.0
        self.c = 0.0
        
        self.set_capacity()


In [47]:
def matrixRandom():
    int i, j;
    for i in range(len(Status.banks)):
        Status.banks[i].connected = 0.0
        for j in range(len(Status.banks)):
            Status.marketBank[i][j] = 0.0

    for i in range(len(Status.banks)):
        if Status.banks[i].connected != 1:
            aux = random.random()
            if aux < Config.RANDOM_CONNECTIVITY:
                j=i
                while i==j:
                    j = int(random.random() * Config.N)
                    if i!=j:
                        Status.marketBank[i][j]=1
                        Status.banks[i].neighbour = j
                        Status.banks[i].connected = 1

    for i in range(len(Status.banks)):
        Status.banks[i].marked = -1
    for i in range(len(Status.banks)):
        if Status.banks[i].marked == -1:
            Status.mark += 1
            matrixRandom_dfs(i)
    
                        
def matrixRandom_dfs(n):
    if (Status.banks[i].marked == -1):
        Status.banks[i].marked = Status.mark
        for i in range(len(Status.banks)):
            if (Status.marketBank[n][i] == 1 || Status.marketBank[i][n] == 1) && Status.banks[i].marked == -1 :
                matrixRandom_dfs(i)

def matrixPreferential():
    float beta[N], interestMin;
  int link[N];
  int k = 0;
  int switching = 0;
  float ref = 0.;
  float fact = 0.;
  float prob = 0.;
  float fitness = 0.;

  liquidityMax = -1000.;
  i_max = -1;
  for (j = 0; j < N; j++) {
    if (liquidity[j] > liquidityMax) {
      liquidityMax = liquidity[j];
      i_max = j;
    }
  }

  /*equityMax = -1000.;
  i_max = -1;
  for(j = 0; j < N; j++){
    if(equity[j] > equityMax){
      equityMax =equity[j];
      i_max = j;
    }
  }*/

  interestMin = 1000.;
  i_max2 = -1;
  for (j = 0; j < N; j++) {
    if (tasso[j] < interestMin) {
      interestMin = tasso[j];
      i_max2 = j;
    }
  }

  for (i = 0; i < N; i++) {
    fit[i] = 0.;
    beta[i] = 0.;
    outgoinglink[i] = 0;
    incominglink[i] = 0;
    link[i] = 0;
  }
  for (j = 0; j < N; j++) {
    fit[j] = (double)(ypsilon * (liquidity[j] / liquidityMax) + (1. - ypsilon) * (interestMin / tasso[j]));
    //fit[j] = (double)(ypsilon*(equity[j]/equityMax)+(1.-ypsilon)*(interestMin/tasso[j]));
    fprintf(out20, "%d  %f %d \n", t, fit[j], n_sim);
    fflush(out20);
  }

  for (i = 0; i < N; i++) {
    for (j = 0; j < N; j++) {
      marketBank[i][j] = 0;
      marketBank[j][i] = 0;
    }
  }
  for (i = 0; i < N; i++) {
    beta[i] = /*(ran2(&idum)*T_inv)+5;*/ T_inv; ///temperatura (inverso) di i  CHECK!
  }
  for (i = 0; i < N; i++) {
    j = i;
    while (j == i) {
      j = ran2( & idum) * N;
    }
    k = vicino[i];
    prob = ran2( & idum);
    fitness = ((fit[j]) - fit[k]); ///aggiungere i costi di transazione???-trans_cost
    ref = beta[i] * fitness;
    fact = (float) 1. / (1. + exp(-ref));

    if (prob < fact && i != j) { //fitness>0 // prob < fact
      switching++;
      marketBank[i][k] = 0;
      marketBank[i][j] = 1;
      vicino[i] = j;

    } else {
      marketBank[i][j] = 0;
      marketBank[i][k] = 1;
      vicino[i] = k;
    }
  }

  for (i = 0; i < N; i++) {
    connected[i] = 1;
  }

  double cazzo, i_maxperc, i_max2perc, nerd, nerd1;

  i_maxperc = 0.;
  i_max2perc = 0.;
  cazzo = 0.;
  cazzo = N;
  nerd = i_max;
  nerd1 = i_max2;
  i_maxperc = (nerd / cazzo);
  i_max2perc = (nerd1 / cazzo);

  fprintf(out13, "%d %lf %d\n", t, i_maxperc, n_sim);
  fflush(out13);

  fprintf(out3, "%d  %lf %d \n", t, i_max2perc, n_sim);
  fflush(out3);

    
    
def findMaxEquity():
    Status.maxEquity = 0.0
    for i in range(len(Status.banks)):
        if (Status.banks[i].equity>Status.maxEquity):
            Status.maxEquity = Status.banks[i].equity
            
def reviewBanksInterbankRate():
    for j in range(len(Status.banks)):
        for i in range(len(Status.banks)):
            if not Status.credit[j][i]:
                Status.bank[j].a = Config.prud * Status.bank[i].asset   +  Config.δ *  Status.bank[i].asset 
                
                Status.bank[j].b = Status.bank[i].probfail * ( Config.α * Status.bank[i].asset -  Status.bank[i].capacity )
                
                Status.bank[j].c = ( 1 - Status.bank[i].probfail ) * Status.bank[i].capacity 
                
                Status.interestInterbank[j][i] = ( Status.bank[j].a - Status.bank[j].b ) / Status.bank[j].c
                if (Status.interestInterbank[j][i]< Config.minInterestInterbank):
                    Status.interestInterbank[j][i] = Config.minInterestInterbank

def reviewBanksRate():
    for i in range(len(Status.banks)):
        Status.banks[i].connected = 0.0
        Status.banks[i].totalRate = 0.0
        
    for j in range(len(Status.banks)):
        for i in range(len(Status.banks)):
            if not (Status.credit[j][i]):                
                Status.banks[j].connected += 1 
                Status.banks[j].totalRate += self.interestInterbank[j][i]
                Status.banks[j].rate = Status.banks[j].totalRate / Status.banks[j].connected
                

In [48]:
for simulation in range(Config.TOT_SIMULATIONS):
    Status.init()
    for t i range(Config.T):
        if (t<10):
            matrixRandom()
        else:
            
            for i in range(len(Status.banks)):
                Status.banks[i].setCapacity()
                findMaxEquity()
                Status.banks[i].determineProbFail()
                
            reviewBanksInterbankRate()
            reviewBanksRate()
            
            matrixPreferential()
                                

In [61]:
Status

__main__.Status

In [ ]:
Config.prud=0